In [78]:
import pandas as pd
import numpy as np 

In [79]:
credits=pd.read_csv('Dataset/tmdb_5000_credits.csv')
movies=pd.read_csv('Dataset/tmdb_5000_movies.csv')

In [80]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [81]:
movies=movies.merge(credits, on='title')
#merging the 2 df 

In [82]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [83]:
#genere #id #keywords #title #overview #vote # cast # crew 
# these are the important features 

In [84]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [85]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [86]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [87]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [88]:
movies.dropna(inplace=True)

In [89]:
movies.duplicated().sum()

np.int64(0)

In [90]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [91]:
# the genere coloumn has data in non python format parse it using ast.literal_eval

In [92]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj) :
     L.append(i['name'])
    return L

    
    

In [93]:
import ast
movies['genres']=movies['genres'].apply(convert)

In [94]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [95]:
movies['keywords']=movies['keywords'].apply(convert)

In [96]:
def convert3(obj):
    L=[]
    count=0
    for i in ast.literal_eval(obj):
        if count!=3:
            L.append(i['name'])
            count+1
        else:
            break
    return L

In [97]:
movies['cast']=movies['cast'].apply(convert3)

In [98]:
import ast

def fetch_director(text):
    L = []
    
    # Only parse if it's a string (e.g. a JSON string)
    if isinstance(text, str):
        try:
            text = ast.literal_eval(text)
        except:
            return []  # Skip if it's malformed
    
    # If it's not a list now, return empty
    if not isinstance(text, list):
        return []

    for i in text:
        if isinstance(i, dict) and i.get('job') == 'Director':
            L.append(i.get('name'))
    
    return L


| Part                | Purpose                                |
| ------------------- | -------------------------------------- |
| `ast.literal_eval`  | Safely parse string to Python object   |
| `isinstance` checks | Validate type before working with data |
| `for` loop          | Go through each crew member            |
| `.get('job')`       | Safely access values from dictionary   |
| `return L`          | Return final list of directors         |

    the .get method is used instead of i[job ] to reduce error 


In [99]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [100]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]


In [101]:

def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [102]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [103]:
movies['tags']=movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [104]:
new=movies.drop(['overview','genres','keywords','cast','crew'], axis=1)
new.head()

,movie_id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,206647,Spectre,"[Action, Adventure, Crime, spy, basedonnovel, ..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dccomics, cri..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction, basedonnov..."


In [105]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()


,movie_id,title,tags
0,19995,Avatar,Action Adventure Fantasy ScienceFiction cultur...
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drugabuse exoti...
2,206647,Spectre,Action Adventure Crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller dccomics crimefigh...
4,49529,John Carter,Action Adventure ScienceFiction basedonnovel m...


VECTORIZATION

In [106]:
!pip install nltk

In [107]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [108]:
new['tags'] = new['tags'].fillna('') 
vectors= cv.fit_transform(new['tags']).toarray()

In [109]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], shape=(5000,))

In [110]:
import nltk 

In [111]:
from nltk.stem.porter import PorterStemmer

In [43]:
ps =PorterStemmer()

In [47]:
def stem(text ):
    y=[]

    for i in text.split():
        y.append(ps.stem(i))
        return ''.join(y)

In [49]:
new['tags']=new['tags'].apply(stem)

In [50]:
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
similarity = cosine_similarity(vectors)

In [63]:
similarity

array([[1.        , 0.08000711, 0.07543143, ..., 0.04956816, 0.        ,
        0.        ],
       [0.08000711, 1.        , 0.11164844, ..., 0.        , 0.        ,
        0.        ],
       [0.07543143, 0.11164844, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.04956816, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]], shape=(4806, 4806))

In [67]:

def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [75]:
recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Amidst the Devil's Wings
Mi America
Ajami
